In [1]:
import os
import json
import os.path as osp


def load_json(json_path):
    with open(json_path, 'r') as f:
        json_data = json.load(f)
    return json_data


data_root = '/data_new/chence/LPFF'
json_files = sorted([i for i in os.listdir(data_root) if i.endswith('.json') and i.startswith('meta_')])
print(json_files)

['meta_1-2.json', 'meta_2-2.json']


In [2]:
merged_data = {}
for idx in range(len(json_files)):
    json_file = json_files[idx]
    with open(os.path.join(data_root, json_file), 'r') as f:
        data = json.load(f)
        print(len(data))
        merged_data.update(data)
print(len(merged_data))

10000
5688
15688


In [4]:
meta_json_file = 'meta.json'
with open(os.path.join(data_root, meta_json_file), 'w') as f:
    json.dump(merged_data, f, indent=4)

In [3]:
landmark_json_file = 'raw_lm.json'
landmark_json_data = load_json(os.path.join(data_root, landmark_json_file))
print(len(landmark_json_data))

19590


In [4]:
import tqdm

lm_data = {}
for k in tqdm.tqdm(landmark_json_data.keys()):
    nk = f'{k[:-3]}.png'
    hid = int(k[-2:])-1
    if nk not in lm_data.keys():
        lm_data[nk] = {}
    lm_data[nk][f'{hid:02d}'] = landmark_json_data[k]

  0%|          | 0/19590 [00:00<?, ?it/s]

100%|██████████| 19590/19590 [00:00<00:00, 274436.01it/s]


In [5]:
import torch
import sys
sys.path.insert(0, '..')
import ChamferDistancePytorch.chamfer2D.dist_chamfer_2D as dist_chamfer_2D
import numpy as np

chamLoss = dist_chamfer_2D.chamfer_2DDist()

Loaded compiled 2D CUDA chamfer distance


In [6]:
import copy

filtered_meta = {}

for merged_key in tqdm.tqdm(merged_data.keys()):
    merged_img = merged_key.split('/')[1]
    saved_lm_idx = []
    for lmk in lm_data[merged_img].keys():
        lmval = lm_data[merged_img][lmk]
        lmk_names = []
        dists = []
        for lmk_ in merged_data[merged_key]['raw']['landmarks'].keys():
            if merged_data[merged_key]['head'][lmk_]['view'] == 'back': continue
            lmval_ = merged_data[merged_key]['raw']['landmarks'][lmk_]
            if lmval_ is None or lmval is None:
                continue
            dist1, dist2, idx1, idx2 = chamLoss(
                torch.from_numpy(np.array(lmval_, dtype=np.float32)).unsqueeze(0).cuda(),
                torch.from_numpy(np.array(lmval, dtype=np.float32)).unsqueeze(0).cuda()
            )
            lmk_names.append(lmk_)
            dists.append((np.mean(dist1.cpu().detach().numpy())+np.mean(dist2.cpu().detach().numpy()))/2.)
        if len(dists) != 0:
            sid = lmk_names[np.argmin(dists)]
            if merged_data[merged_key]['head'][sid]['view'] != 'back':
                dists = np.array(dists)
                saved_lm_idx.append(sid)
    saved_lm_idx = sorted(set(saved_lm_idx))
    if len(saved_lm_idx) != 0:
        filtered_meta[merged_key] = copy.deepcopy(merged_data[merged_key])
        filtered_meta[merged_key]['raw']['landmarks'] = {}
        filtered_meta[merged_key]['head'] = {}
        for slmidx, imkey in enumerate(saved_lm_idx):
            filtered_meta[merged_key]['raw']['landmarks'][f'{slmidx:02d}'] = merged_data[merged_key]['raw']['landmarks'][imkey]
            filtered_meta[merged_key]['head'][f'{slmidx:02d}'] = merged_data[merged_key]['head'][imkey]

100%|██████████| 15688/15688 [00:19<00:00, 801.98it/s] 


In [7]:
len(filtered_meta)

14293

In [8]:
import tqdm

for k in tqdm.tqdm(filtered_meta.keys()):
    for hk in filtered_meta[k]['head'].keys():
        if filtered_meta[k]['head'][hk]['view'] == 'back':
            print(k, hk)

100%|██████████| 14293/14293 [00:00<00:00, 272853.49it/s]


In [9]:
meta_hq_json_file = 'meta_hq.json'
with open(os.path.join(data_root, meta_hq_json_file), 'w') as f:
    json.dump(filtered_meta, f, indent=4)